## Exercise 
創建一個RNN網路,能夠預測下一個數字


In [1]:
# %load simpleRNN.py
'''
 RNN model type: Many to one
 here, we use sequential model to create a single output tensor,
 For multi-output layers, use the functional API.
 https://keras.io/guides/functional_api/
'''

from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Bidirectional
import numpy as np

In [2]:


# Function to genate the input
# You can try with different input
def sequence_generator(typeof):
	# [0, 10, 20, ...]
	if typeof == 'plusten':
		sequence = [i for i in range(0, 1500, 10)]
	# [2, 4, 6, ...]
	elif typeof == 'double':
		sequence = [i*2 for i in range(1, 250)]
	# [1, 4, 9, 25, ...]
	elif typeof == 'power':
		sequence = [i**2 for i in range(1, 100)]
	return sequence


#  To separate the sequence into input and output ([X] [y])
# For example [1, 3, 5, 7, 9, 11] becomes
# [1, 3, 5] --> [7]
# [3, 5, 7] --> [9]
# [5, 7, 9] --> [11] 
def separate_sequence(sequence, seq_len):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + seq_len
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)
 




## Generate the input sequence

In [3]:
time_step = 4
n_features = 1

t = 'power'
sq = sequence_generator(t)
print (sq)

# split into samples
X, y = separate_sequence(sq,time_step)

# reshape the input into [batch, timesteps, features] to be readable to the network
print(X[:5])
print(y[:5])
X = X.reshape((X.shape[0], X.shape[1], n_features))
print('X.shape',X.shape)
print('y.shape',y.shape)

[1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121, 144, 169, 196, 225, 256, 289, 324, 361, 400, 441, 484, 529, 576, 625, 676, 729, 784, 841, 900, 961, 1024, 1089, 1156, 1225, 1296, 1369, 1444, 1521, 1600, 1681, 1764, 1849, 1936, 2025, 2116, 2209, 2304, 2401, 2500, 2601, 2704, 2809, 2916, 3025, 3136, 3249, 3364, 3481, 3600, 3721, 3844, 3969, 4096, 4225, 4356, 4489, 4624, 4761, 4900, 5041, 5184, 5329, 5476, 5625, 5776, 5929, 6084, 6241, 6400, 6561, 6724, 6889, 7056, 7225, 7396, 7569, 7744, 7921, 8100, 8281, 8464, 8649, 8836, 9025, 9216, 9409, 9604, 9801]
[[ 1  4  9 16]
 [ 4  9 16 25]
 [ 9 16 25 36]
 [16 25 36 49]
 [25 36 49 64]]
[25 36 49 64 81]
X.shape (95, 4, 1)
y.shape (95,)


## create model and train model

In [10]:
model = Sequential()
#model.add(Bidirectional(LSTM(64, activation='relu'), input_shape=(time_step, n_features)))
model.add(LSTM(256, activation='relu',input_shape=(time_step, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
model.summary()
# fit model
model.fit(X, y, epochs=400, batch_size =32,verbose=0)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 256)               264192    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 264,449
Trainable params: 264,449
Non-trainable params: 0
_________________________________________________________________


## Test

In [11]:
# Input a sequence and predict
x_input = array([39,49,59,69])
x_input = x_input.reshape((1, time_step, n_features))
yhat = model.predict(x_input, verbose=0)
print(np.round(yhat))

[[99.]]


In [12]:
print(X[:10])
yhat = model.predict(X[:10], verbose=0)
np.round(yhat)

[[[  1]
  [  4]
  [  9]
  [ 16]]

 [[  4]
  [  9]
  [ 16]
  [ 25]]

 [[  9]
  [ 16]
  [ 25]
  [ 36]]

 [[ 16]
  [ 25]
  [ 36]
  [ 49]]

 [[ 25]
  [ 36]
  [ 49]
  [ 64]]

 [[ 36]
  [ 49]
  [ 64]
  [ 81]]

 [[ 49]
  [ 64]
  [ 81]
  [100]]

 [[ 64]
  [ 81]
  [100]
  [121]]

 [[ 81]
  [100]
  [121]
  [144]]

 [[100]
  [121]
  [144]
  [169]]]


array([[ 25.],
       [ 36.],
       [ 49.],
       [ 64.],
       [ 81.],
       [101.],
       [121.],
       [144.],
       [169.],
       [196.]], dtype=float32)